# Merging the NIRs data with text columns

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm

# all data
data_path = '../data'

# nirs data
nirs_data_sub1 = os.path.join(data_path, 'neuro/nirs_sub1.csv')
nirs_data_sub2 = os.path.join(data_path, 'neuro/nirs_sub2.csv')

# text data
ceda_data_path = os.path.join(data_path, 'results/ceda-results.csv')
########## may need to change . . . I need dyads->speakers data.
meta_data_path = os.path.join(data_path, 'meta_data/CC_Post_105dyads.csv')

## Creating NIRs, Transcript Specific CSVs

In [ ]:
id_col = 'Subject'
for sdf in pd.read_csv(nirs_data_sub1, chunksize=1000):
    
    for dyad in sdf[id_col].unique():
        f = os.path.join(data_path, 'unified_neuro', str(dyad)+'.csv')
        
        sub = sdf.loc[sdf[id_col].isin([dyad])].copy()
        # sub = sub.rename(columns={col: 'x_'+col for col in sub.columns})
        
        if os.path.exists(f):
            sub.to_csv(f, encoding='utf-8', mode='a', header=False, index=False)
        else:
            sub.to_csv(f, encoding='utf-8', index=False)

In [ ]:
id_col = 'Subject'
current_df = []
for sdf in pd.read_csv(nirs_data_sub2, chunksize=1000):
    
    for dyad in sdf[id_col].unique():
        f = os.path.join(data_path, 'unified_neuro', str(dyad)+'_2.csv')
        
        sub = sdf.loc[sdf[id_col].isin([dyad])].copy()
        # sub = sub.rename(columns={col: 'x_'+col for col in sub.columns})
        
        if os.path.exists(f):
            sub.to_csv(f, encoding='utf-8', mode='a', header=False, index=False)
        else:
            sub.to_csv(f, encoding='utf-8', index=False)

Merging the data

In [ ]:
unified_neuro_home = os.path.join(data_path,'unified_neuro')
files = [f for f in os.listdir(unified_neuro_home) if (not f.endswith('_2.csv')) and (not f.startswith('._'))]

for f in tqdm(files):
    df = pd.read_csv(os.path.join(unified_neuro_home, f))
    s2df = pd.read_csv(os.path.join(unified_neuro_home, f.replace('.csv', '_2.csv')))
    channel_columns = [col for col in list(df) if 'ch_' in col]
    df[channel_columns] = (df[channel_columns] - s2df[channel_columns]).abs()
    
    df.to_csv(os.path.join(unified_neuro_home, f), index=False, encoding='utf-8')
    os.remove(os.path.join(unified_neuro_home, f.replace('.csv', '_2.csv')))